## Generate HeFESTo lookup table for ASPECT

In [1]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
import shilofue.PostHefesto as PostHefesto

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities

# if we have set up a project directory, use it for the following paths
project_dir = os.environ["HeFESTo_DIR"]

### Convert fort.56 file from HeFESTo to importable table to ASPECT for the T, P table

#### 1. Assign the path to the "fort.56" file below.

#### 2. Assign the path of the output.

For ASPECT to import the table, we need both the correct format of the table header and the two dimentional variables in the first, second columns.
To Do the conversion, the following flocks are needed.

First the P-T table.

In [2]:
# P T table
project_dir = os.environ["HeFESTo_DIR"]

case_dir = os.path.join(project_dir, "hefesto_pyrolite_P_T")

assert(os.path.isdir(case_dir))

#### First we assembles sub tables from parallel runs

The HeFESTo code is parallel with slurm, so the fort.56 files are distributed by the threads.
The first step is to find all the fort.56 outputs and combine them into a collective one.

The output file will be automatically saved to this location:

    output/fort.56

In [ ]:
# assemble
# PostHefesto.AssembleParallelFiles(case_dir)

#### Generate table for ASPECT

Before the table is ready for ASPECT, we check that the data aligned in the two dimentions and then we properly add a header to the table

This might not work at first try.
The outputs from HeFESTo could be messed up if there is a long string, like this example:

    3.5274338280952606-2782772.4668332533910871

We need to first add a blank in between:

    3.5274338280952606 -2782772.4668332533910871

The options of 

    exchange_dimension=True

would exchange the 1st and 2nd dimensions of the output. In this case, I use it to first output the temperature

In [ ]:
# generate table

input_file = os.path.join(case_dir, "output/fort.56")
# output_file = os.path.join(case_dir, "output/pyrolite_PT.txt")
output_file = os.path.join(case_dir, "output/pyrolite_PT_small.txt")

interval1=284; interval2=2560

exchange_dimension=True

# assert input file exists
assert(os.path.isfile(input_file))

if (os.path.isfile(output_file)):  # remove old files
    os.remove(output_file)

# read input
LookupTable = PostHefesto.LOOKUP_TABLE()
LookupTable.ReadRawFort56(input_file)

# process output
field_names = ['Pressure', 'Temperature', "Entropy", 'Density', 'Thermal_expansivity', 'Isobaric_heat_capacity',\
                'VP', 'VS', 'Enthalpy', "most abundant phase"]
maph_lookup = ["pv"]
LookupTable.Process(field_names, output_file, interval1=interval1, interval2=interval2, file_type="structured",\
                    fix_coordinate_minor=True, exchange_dimension=exchange_dimension, maph_lookup=maph_lookup)

# assert something 
assert(os.path.isfile(output_file))

ReadRawFort56: reading file


ReadRawFort56: data dimension:  (729885, 18)
Process begin indexing
Process finish indexing
OutputPerplexTable: Outputing Data
Outputing fields: ['Pressure', 'Temperature', 'Entropy', 'Density', 'Thermal_expansivity', 'Isobaric_heat_capacity', 'VP', 'VS', 'Enthalpy', 'most abundant phase']
first dimension:  10 , second dimension:  1 , size: 10
New file generated: /mnt/lochy/ASPECT_DATA/HeFESTo/hefesto_pyrolite_P_T/output/pyrolite_PT_small.txt


### Convert fort.56 file from HeFESTo to importable table to ASPECT for the P, S table

In [ ]:
# P S table
case_dir = os.path.join(project_dir, "hefesto_pyrolite_P_S")
assert(os.path.isdir(case_dir))

In [ ]:
# assemble
PostHefesto.AssembleParallelFiles(case_dir)

The options of 

    exchange_dimension=True

would exchange the 1st and 2nd dimensions of the output. In this case, I use it to first output the entropy

In [ ]:
# generate table

interval1=10; interval2=5

input_file = os.path.join(case_dir, "output/fort.56")
output_file = os.path.join(case_dir, "output/pyrolite_PS_10252024.txt")

# exchange 1st and 2nd dimensions
exchange_dimension=True

# assert input file exists
assert(os.path.isfile(input_file))

if (os.path.isfile(output_file)):  # remove old files
    os.remove(output_file)

# read input
LookupTable = PostHefesto.LOOKUP_TABLE()
LookupTable.ReadRawFort56(input_file)

# process output
# note: now the output for "most abundant phase" has a formatting error
field_names = ['Pressure', 'Entropy', 'Temperature', 'Density', 'Thermal_expansivity', 'Isobaric_heat_capacity', 'VP', 'VS', 'Enthalpy', "most abundant phase"]
maph_lookup = ["pv"]
LookupTable.Process(field_names, output_file, interval1=interval1, interval2=interval2, second_dimension="Entropy", file_type="structured",\
                    fix_coordinate_minor=True, exchange_dimension=exchange_dimension, maph_lookup=maph_lookup)

# assert something 
assert(os.path.isfile(output_file))

In [ ]:
# LookupTable.data[0, 16]
LookupTable.data[0, 17]